In [1]:
import os ; from pprint import pformat ; import flopy ; import git ; import matplotlib.pyplot as plt ; import numpy as np; 
from flopy.plot.styles import styles ; import pandas as pd; from pathlib import Path ; import modflow_devtools.misc 
from modflow_devtools.misc import get_env, timed    # pip install modflow_devtools

In [2]:
parameter_units = {"dispersivity": "$m$","retardation": "unitless","decay": "$d^{-1}$",}  ; length_units = "meters" ; time_units = "days"
exe_name_mt = 'E:\\15_REPOS\\00_BETAMI\\w10_______others\\bin\\mt3d-usgs_1.1.0_64'
exe_name_mu = 'E:\\15_REPOS\\00_BETAMI\\w10_______others\\bin\\mf6'
workspace = Path('A_p01_V02') ; workspace.mkdir(exist_ok=True)

In [3]:
nper = 1    ; nlay = 1     ; ncol = 101      ; nrow = 1      ; delr = 10.0  ; delc = 1.0  
top  = 0.0  ; botm = -1.0  ; prsity = 0.25   ; perlen = 2000 ; k11 = 1.0    ; k33 = k11 
laytyp = 1  ; nstp = 100.0 ; dt0 = perlen / nstp  ; Lx = (ncol - 1) * delr  ; v = 0.24  ; q = v * prsity   ; h1 = q * Lx

strt   = np.zeros((nlay, nrow, ncol), dtype=float)  ; strt[0, 0, 0] = h1    ;  l = 1000.0             ; icelltype = 1  # (l) Needed for plots
ibound = np.ones ((nlay, nrow, ncol), dtype=int)    ; ibound[0, 0, 0] = -1  ;  ibound[0, 0, -1] = -1  ; mixelm = 0 ;    rhob = 0.25 ; sp2 = 0.0  
sconc = np.zeros ((nlay, nrow, ncol), dtype=float)  ; dmcoef = 0.0          ; nouter, ninner = 100, 300                                  

hclose, rclose, relax = 1e-6, 1e-6, 1.0       ; ttsmult = 1.0 ; dceps = 1.0e-5  ; nplane = 1  ; npl = 0  ; nph = 4  ; npmin = 0  # HMOC
npmax = 8  ; nlsink = nplane  ; npsink = nph  ; tdis_rc = []                                              

tdis_rc.append((perlen, nstp, 1.0))  ; chdspd = [[(0, 0, 0), h1], [(0, 0, ncol - 1), 0.0]] ; c0 = 1.0 ; cncspd = [[(0, 0, 0), c0]]

c0 = 1.0
icbund = np.ones((nlay, nrow, ncol), dtype=int)      ; icbund [0, 0, 0] = -1
sconc = np.zeros((nlay, nrow, ncol), dtype=float)    ; sconc  [0, 0, 0] = c0   ; dispersivity = 10.0  ; retardation = 5.0  ; decay  = 0.002

In [4]:
gwfname = 'gwf_A' ;  name = 'name'

sim    = flopy.mf6.MFSimulation(sim_name= 'sim_A' , sim_ws=workspace, exe_name=exe_name_mu)
tdis   = flopy.mf6.ModflowTdis(sim, nper=nper, perioddata=tdis_rc, time_units=time_units)
gwf    = flopy.mf6.ModflowGwf(sim, modelname=gwfname, save_flows=True,model_nam_file=f"{gwfname}.nam")
imsgwf = flopy.mf6.ModflowIms(sim,print_option="SUMMARY",outer_dvclose=hclose,outer_maximum=nouter,
                              under_relaxation="NONE",inner_maximum=ninner,inner_dvclose=hclose,rcloserecord=rclose,
                              linear_acceleration="CG",scaling_method="NONE",reordering_method="NONE",relaxation_factor=relax,
                              filename=f"{gwfname}.ims")                                       
sim.register_ims_package(imsgwf, [gwf.name])
dis = flopy.mf6.ModflowGwfdis(gwf,length_units=length_units,nlay=nlay,nrow=nrow,ncol=ncol,delr=delr,delc=delc,top=top,botm=botm,
                              idomain=np.ones((nlay, nrow, ncol), dtype=int),filename=f"{gwfname}.dis")
npf = flopy.mf6.ModflowGwfnpf(gwf,save_flows=False,icelltype=icelltype,k=k11,k33=k33,save_specific_discharge=True,filename=f"{gwfname}.npf")
fic = flopy.mf6.ModflowGwfic(gwf, strt=strt, filename=f"{gwfname}.ic")
chd = flopy.mf6.ModflowGwfchd(gwf,maxbound=len(chdspd),stress_period_data=chdspd,save_flows=False,pname="CHD-1",filename=f"{gwfname}.chd")
foc = flopy.mf6.ModflowGwfoc(gwf,head_filerecord=f"{gwfname}.hds",budget_filerecord=f"{gwfname}.cbc",
                             headprintrecord=[("COLUMNS", 10, "WIDTH", 15, "DIGITS", 6, "GENERAL")],
                                saverecord=[("HEAD", "LAST"), ("BUDGET", "LAST")],
                                    printrecord=[("HEAD", "LAST"), ("BUDGET", "LAST")])                            
gwtname = 'gwt_A' 
gwt = flopy.mf6.MFModel(sim,model_type="gwt6"  , modelname=gwtname  , model_nam_file=f"{gwtname}.nam")      ; gwt.name_file.save_flows = True
imsgwt = flopy.mf6.ModflowIms(sim,print_option="SUMMARY",outer_dvclose=hclose,outer_maximum=nouter,under_relaxation="NONE",
                              inner_maximum=ninner,inner_dvclose=hclose,rcloserecord=rclose,linear_acceleration="BICGSTAB", scaling_method="NONE",
                              reordering_method="NONE",relaxation_factor=relax,filename=f"{gwtname}.ims",); sim.register_ims_package(imsgwt, [gwt.name])
dis = flopy.mf6.ModflowGwtdis(gwt,nlay=nlay,nrow=nrow,ncol=ncol,delr=delr,delc=delc,top=top,botm=botm,idomain=1,filename=f"{gwtname}.dis")
ic = flopy.mf6.ModflowGwtic(gwt, strt=sconc, filename=f"{gwtname}.ic")
    
if mixelm == 0:
    scheme = "UPSTREAM"
elif mixelm == -1:
    scheme = "TVD"
else:
    raise Exception()  
    
adv = flopy.mf6.ModflowGwtadv(gwt, scheme=scheme, filename=f"{gwtname}.adv")
    
if dispersivity != 0:
    dsp = flopy.mf6.ModflowGwtdsp(gwt,xt3d_off=True,alh=dispersivity,ath1=dispersivity,filename=f"{gwtname}.dsp",)
if retardation != 1.0:
    sorption = "linear"        ; bulk_density = rhob    ; kd = (retardation - 1.0) * prsity / rhob  # prsity & rhob defined in
else:  
    sorption = None            ; bulk_density = None    ; kd = None
if decay != 0.0:
    first_order_decay = True   ; decay_arg = decay
else:
    first_order_decay = False  ; decay_arg = None
mst = flopy.mf6.ModflowGwtmst(gwt,porosity=prsity,sorption=sorption,bulk_density=bulk_density,distcoef=kd,
                              first_order_decay=first_order_decay,decay=decay_arg,decay_sorbed=decay_arg,filename=f"{gwtname}.mst")

cnc = flopy.mf6.ModflowGwtcnc(gwt,maxbound=len(cncspd),stress_period_data=cncspd,save_flows=False,pname="CNC-1",filename=f"{gwtname}.cnc",)
ssm = flopy.mf6.ModflowGwtssm(gwt, sources=[[]], filename=f"{gwtname}.ssm")
toc = flopy.mf6.ModflowGwtoc(gwt,budget_filerecord=f"{gwtname}.cbc",concentration_filerecord=f"{gwtname}.ucn",
                             concentrationprintrecord=[("COLUMNS", 10, "WIDTH", 15, "DIGITS", 6, "GENERAL")],
                                saverecord=[("CONCENTRATION", "LAST"), ("BUDGET", "LAST")],
                                    printrecord=[("CONCENTRATION", "LAST"), ("BUDGET", "LAST")],)
flopy.mf6.ModflowGwfgwt(sim,exgtype="GWF6-GWT6",exgmnamea=gwfname,exgmnameb=gwtname,filename=f"{name}.gwfgwt")

sim.write_simulation(silent=True)
sim.run_simulation(silent=True) 

(True, [])

In [5]:
hobj = gwf.output.head()                 ; hds      = hobj.get_data()                      ; print('H (2000d) = ', hds      .shape)
ucnobj_mf6 = gwt.output.concentration()  ; conc_mf6 = ucnobj_mf6.get_alldata()             ; print('C (2000d) = ', conc_mf6 .shape)
cobj = gwf.output.budget()               ; spd      = cobj.get_data(text="DATA-SPDIS")[0]  ; print('Q (2000d) = ', spd      .shape)

H (2000d) =  (1, 1, 101)
C (2000d) =  (1, 1, 1, 101)
Q (2000d) =  (101,)


In [6]:
reSHP_H  = hds.reshape (hds.shape[0], -1)       ; then_H = pd.DataFrame(reSHP_H) ; then_H.to_csv("ex-p1_H.csv") ; print("H_2D_All=",then_H.shape) 
reSHP_C  = conc_mf6.reshape (hds.shape[0], -1)  ; then_C = pd.DataFrame(reSHP_C) ; then_C.to_csv("ex-p1_C.csv") ; print("C_2D_All=",then_H.shape) 
then_Q   = pd.DataFrame(spd)                                                     ; then_Q.to_csv("ex-p1_Q.csv")

H_2D_All= (1, 101)
C_2D_All= (1, 101)


In [7]:
# Just Plot ( C x n )